In [1]:
using LinearRegression
using Statistics
using CairoMakie
using DelimitedFiles
using Megafauna
using HypothesisTests
using Distances
using StatsBase
include("../src/SegmentDistances.jl")
include("../src/DPC.jl")

function identify_transitions(X, cps)
    N = length(cps) - 1
    t1, t2, t3, t4 = cps[1], cps[2], cps[3], cps[4]
    predecessor = X[t1:t2]
    current = X[t2:t3]
    successor = X[t3:t4]
    m_pred, σ_pred = median(predecessor), std(predecessor)
    m_curr, σ_curr = median(current), std(current)
    m_succ, σ_succ = median(successor), std(successor)
    transition_labels = zeros(N)
    mstable_stats = zeros(N)
    trans_stats = zeros(N)
    for i = 2:N-1
        metalike = 0
        translike = 0
        L = length(current)
        for (idx, x) in enumerate(current)
            λ = idx / (L + 1)
            μ = λ * median(successor) + (1 - λ) * median(predecessor)
            ρ = λ * std(successor) + (1 - λ) * std(predecessor)
            metalike -= log(2* σ_curr) + (abs(x - m_curr) / σ_curr)
            translike -= log(2 * ρ) + (abs(x - μ) / ρ)
        end
        mstable_stats[i] = metalike 
        trans_stats[i] = translike
        transition_labels[i] = metalike > translike ? 0 : 1
        if i == N - 1
            break
        end
        predecessor = current
        current = successor
        successor = X[cps[i+2]:cps[i+3]]
        m_pred, σ_pred = m_curr, σ_curr
        m_curr, σ_curr = m_succ, σ_succ
        m_succ, σ_succ = median(successor), std(successor)
    end
    return (transition_labels, mstable_stats, trans_stats)
end

function kstest(X, cps, segment_labels)
    N = length(cps) - 1
    laplace_pvalues = []
    normal_pvalues = []
    mstable_lengths = []
    n_mstable = 0
    
    for i=1:N
        if segment_labels[i] > 0
            continue
        end
        n_mstable += 1
        segment = X[cps[i]:cps[i+1]]
        push!(mstable_lengths, length(segment))
        Z = fit_mle(Laplace, segment)
        Zprime = fit_mle(Normal, segment)
        push!(laplace_pvalues, pvalue(ExactOneSampleKSTest(segment, Z)))
        push!(normal_pvalues, pvalue(ExactOneSampleKSTest(segment, Zprime)))
    end
    laplace_reject_rate = 1 - (count(>(0.05), laplace_pvalues)) / n_mstable 
    normal_reject_rate = 1 - (count(>(0.05), normal_pvalues)) / n_mstable 

    return (laplace_pvalues, normal_pvalues, laplace_reject_rate, normal_reject_rate)
    
end

function change_point_F1(cps, true_cps; tolerance=50)
    println(tolerance)
    positives = [any(abs.(true_cps .- x) .< tolerance) for x in cps]
    true_positives = count(==(1), positives)
    # if the closest detected change point is more than tolerance away, you missed the change point
    false_positives = count(==(0), positives)
    negatives = [minimum(abs.(cps .- x)) .> tolerance for x in true_cps] 
    false_negatives = count(==(1), negatives)
    precision = true_positives / (true_positives + false_positives)
    recall = true_positives / (true_positives + false_negatives)
    F1 = 2 * (precision * recall) / (recall + precision)
    return (recall, precision, F1, positives, negatives)
end

function transition_F1(point_labels, true_labels)
    arr = cat(point_labels, true_labels, dims=2)
    TP, FP, FN = 0, 0, 0
    for row in eachrow(arr)
        if row[1] == 1 && row[2] == 1
            TP += 1
        elseif row[1] == 1 && row[2] == 0
            FP += 1
        elseif row[1] == 0 && row[2] == 1
            FN += 1
        else
            continue
        end
    end
    precision = TP / (TP + FP)
    recall = TP / (TP + FN)
    F1 = 2 * precision * recall / (precision + recall)
    return (recall, precision, F1)
end





function plot_trajectories_with_change_points(
        X,
        bart_cps, bart_point_labels, bart_positives, bart_negatives,
        mf_cps, mf_point_labels, mf_positives, mf_negatives,
        true_cps, true_point_labels)
    fig_size = (1000,600)
    fig = Figure(size=fig_size)
    cmap=Makie.Categorical(:darktest)
    cmap_segs = Makie.Categorical(:darktest)
    
    ymin, ymax, ystep = minimum(X), maximum(X), floor((maximum(X) - minimum(X)) / 5)
    ax1 = Axis(fig[1,1])
    ax2 = Axis(fig[2,1])
    ax1.title="Toy Trajectory Segmented via BarT"
    ax2.title="Toy Trajectory Segmented via Megafauna"
    ax1.xlabel = ax2.xlabel = "t"
    ax1.ylabel = ax2.ylabel = "x(t)"
    ax1.xticks = ax2.xticks = 0:500:length(X)
    
    ax1.yticks = ax2.yticks = ymin:ystep:ymax
    ax1.xtickformat = ax2.xtickformat = "{:.0f}"
    ax1.ytickformat = ax2.ytickformat = "{:.0f}"


    mf_colors = map(b -> b > 0 ? :gray : :black, mf_point_labels)
    bart_colors = map(b -> b > 0 ? :gray : :black, bart_point_labels)
    mf_positives_colors = map(b -> b > 0 ? :green : :red, mf_positives)
    bart_positives_colors = map(b -> b > 0 ? :green : :red, bart_positives)
    mf_negatives_colors = map(b -> b > 0 ? :orange : :white, mf_negatives)
    bart_negatives_colors = map(b -> b > 0 ? :orange : :white, bart_negatives)


    scatter!(ax1, 1:length(X), X[:,1], color=bart_colors, markersize=2)
    scatter!(ax2, 1:length(X), X[:,1], color=mf_colors, markersize=2)

    println(size(bart_negatives))
    vlines!(ax1, true_cps[:,1], color=bart_negatives_colors[:,1], linestyle=:dashdot, colormap=cmap)
    vlines!(ax1, bart_cps[:,1], color=bart_positives_colors[:,1], linestyle=:solid, colormap=cmap)
    vlines!(ax2, true_cps[:,1], color=mf_negatives_colors[:,1], linestyle=:dashdot, colormap=cmap)
    vlines!(ax2, mf_cps[:,1], color=mf_positives_colors[:,1], linestyle=:solid, colormap=cmap)
    
    elem_1 = [LineElement(color = :green, linestyle = :solid)]
    elem_2 = [LineElement(color = :red, linestyle = :solid)]
    elem_3 = [LineElement(color = :orange, linestyle = :dashdot)]
    elem_4 = [LineElement(color = :black, linestyle = nothing)]
    elem_5 = [LineElement(color = :gray, linestyle = nothing)]
    
    axislegend(ax1,
        [elem_1, elem_2, elem_3, elem_4, elem_5],
        ["True Positive", "False Positive", "False Negative","Metastable", "Transition"],
        patchsize = (5, 5), rowgap = 1, position=:lt)
    return fig
    
end

function color_by_frequency(l)
    # Get frequency counts of each label
    freq = countmap(l)
    
    # Sort labels by frequency (most frequent first)
    sorted_labels = sort(collect(keys(freq)), by=x->freq[x], rev=true)
    println(freq)
    
    # Create a color map with blue, green, red for top 3 frequencies
    colors = Dict(
        sorted_labels[1] => :blue,
        sorted_labels[2] => :green,
        sorted_labels[3] => :red
    )
    
    # For any additional labels, they get gray
    for i in 4:length(sorted_labels)
        colors[sorted_labels[i]] = :gray
    end
    
    # Plot with the appropriate colors

    return color=[colors[label] for label in l]
end

function plot_clustered_trajectories(
        X,
        bart_cps, bart_point_labels, #bart_positives, bart_negatives,
        mf_cps, mf_point_labels, #mf_positives, mf_negatives,
        true_cps, true_point_labels)
    fig_size = (1000,600)
    fig = Figure(size=fig_size)
    cmap=Makie.Categorical(:darktest)
    cmap_segs = Makie.Categorical(:darktest)
    
    ymin, ymax, ystep = minimum(X), maximum(X), floor((maximum(X) - minimum(X)) / 5)
    ax1 = Axis(fig[1,1])
    ax2 = Axis(fig[2,1])
    ax3 = Axis(fig[3,1])
    ax1.title="Toy Trajectory Segmented via BarT"
    ax2.title="Toy Trajectory Segmented via Megafauna"
    ax3.title="Toy Trajectory Segmented via Ground Truth"

    ax1.xlabel = ax2.xlabel = ax3.xlabel = "t"
    ax1.ylabel = ax2.ylabel = ax3.ylabel ="x(t)"
    ax1.xticks = ax2.xticks = ax3.xticks =0:500:length(X)
    
    ax1.yticks = ax2.yticks = ax3.yticks =ymin:ystep:ymax
    ax1.xtickformat = ax2.xtickformat = ax3.xtickformat = "{:.0f}"
    ax1.ytickformat = ax2.ytickformat = ax3.ytickformat = "{:.0f}"

    scatter!(ax1, 1:length(X), X[:,1], color=color_by_frequency(bart_point_labels[:,1]), colormap=:rainbow, markersize=2)
    scatter!(ax2, 1:length(X), X[:,1], color=color_by_frequency(mf_point_labels[:,1]), colormap=:rainbow, markersize=2)
    scatter!(ax3, 1:length(X), X[:,1], color=color_by_frequency(true_point_labels[:,1]), colormap=:rainbow, markersize=2)


    #vlines!(ax1, true_cps[:,1], color=bart_negatives_colors[:,1], linestyle=:dashdot, colormap=cmap)
    #vlines!(ax1, bart_cps[:,1], color=bart_positives_colors[:,1], linestyle=:solid, colormap=cmap)
    #vlines!(ax2, true_cps[:,1], color=mf_negatives_colors[:,1], linestyle=:dashdot, colormap=cmap)
    #vlines!(ax2, mf_cps[:,1], color=mf_positives_colors[:,1], linestyle=:solid, colormap=cmap)
    
    elem_1 = [LineElement(color = :blue, linestyle = :solid)]
    elem_2 = [LineElement(color = :green, linestyle = :solid)]
    elem_3 = [LineElement(color = :red, linestyle = :solid)]
    #elem_4 = [LineElement(color = :black, linestyle = nothing)]
    #elem_5 = [LineElement(color = :gray, linestyle = nothing)]
    
    Legend(fig[2, 2], [elem_1, elem_2, elem_3], ["Largest pop.", "2nd Largest Pop.", "3rd Largest pop."])
    return fig
    
end

function cpq_score(cps, true_cps)
    return norm([minimum(abs.(true_cps .- x)) for x in cps])
end

function plot_pvalue_hist(pvalues1, pvalues2, method_name)
    fig = Figure(size=(1000, 500))
    ax1 = Axis(fig[1,1])
    ax2 = Axis(fig[1,2])
    ax1.title="Distribution of P Values for KS Test \n Segmentation Method: $(method_name), Prior: Laplace"
    ax2.title="Distribution of P Values for KSTest \n Segmentation Method: $(method_name), Prior: Gaussian"
    ax1.xticks = ax2.xticks = 0:0.05:1.
    ax1.xtickformat = ax2.xtickformat = "{:.2f}"
    ax1.xticklabelrotation = ax2.xticklabelrotation=45.0

    hist!(ax1, pvalues1, bins=25)
    hist!(ax2, pvalues2, bins=25)
    return fig
    
end


function cp_pipeline(σ, trans; tolerance=1)
    # laod data and ID cps
    X = readdlm("../data/Toy_Model_Trajectories/sigma_$(σ)_transition_$(trans).txt")
    bart_cps = convert(Array{Int32}, readdlm("../data/Toy_Model_Trajectories/sigma_$(σ)_transition_$(trans)_cps.txt"))
    true_cps = convert(Array{Int32}, readdlm("../data/Toy_Model_Trajectories/sigma_$(σ)_transition_$(trans)_GroundTruth.txt"))
    true_segment_lenghts = [true_cps[i+1] - true_cps[i] + 1 for i=1:length(true_cps) - 1]
    windowsize = minimum(true_segment_lenghts[true_segment_lenghts .> 2 * maximum([trans 2])])
    println(windowsize)
    quant = 0.5
    println(quant)
    mf_deriv = compute_metric_wasserstein_derivative(X, windowsize, sqeuclidean)[:,1]
    mf_cps = compute_change_points(X, quant, windowsize)
    bart_cps[1] = 1
    true_cps[1] = 1
    #println("Megafauna Relative Error: $(norm(true_cps - mf_cps) / norm(true_cps))")
    println(cpq_score(bart_cps, true_cps))
    println(cpq_score(mf_cps, true_cps))
    #println("Megafauna Absolute Error: $(norm(true_cps - mf_cps))")


    # identify transitions
    segment_transition_labels, segment_mstable_stats, segment_trans_stats = identify_transitions(X, true_cps);
    mf_segment_transition_labels, mf_segment_mstable_stats, mf_segment_trans_stats = identify_transitions(X, mf_cps);
    bart_segment_transition_labels, bart_segment_mstable_stats, bart_segment_trans_stats = identify_transitions(X, bart_cps);

    # label points
    point_labels = label_series(X, true_cps, segment_transition_labels);
    bart_point_labels = convert(Vector{Float64},label_series(X, bart_cps, bart_segment_transition_labels));
    mf_point_labels = convert(Vector{Float64}, label_series(X, mf_cps, mf_segment_transition_labels));
        
    # analyze segments with KS Test, return distributions of p-values
    mf_pdata = kstest(X, mf_cps, mf_segment_transition_labels)
    bart_pdata = kstest(X, bart_cps, bart_segment_transition_labels)

    mf_pvalue_hist = plot_pvalue_hist(mf_pdata[1], mf_pdata[2], "Megafauna")
    save("megafauna-pvalues-$(σ)-$(trans).png", mf_pvalue_hist)
    bart_pvalue_hist = plot_pvalue_hist(bart_pdata[1], bart_pdata[2],  "BarT")
    save("bart-pvalues-$(σ)-$(trans).png", bart_pvalue_hist)

    # compute F1 scores for change points
    bart_cp_recall, bart_cp_precision, bart_cp_F1, bart_positives, bart_negatives = change_point_F1(bart_cps, true_cps, tolerance=tolerance)
    mf_cp_recall, mf_cp_precision, mf_cp_F1, mf_positives, mf_negatives = change_point_F1(mf_cps, true_cps, tolerance=tolerance)
    println("*************Change Point F1**************")
    println("BarT")
    println(bart_cp_recall, " ",bart_cp_precision, " ",bart_cp_F1)
    println("MF")
    println(mf_cp_recall," ", mf_cp_precision," ", mf_cp_F1)
    # compute F1 scores for metastable vs transition
    bart_transition_recall, bart_transition_precision, bart_transition_F1 = transition_F1(bart_point_labels, point_labels)
    mf_transition_recall, mf_transition_precision, mf_transition_F1 = transition_F1(mf_point_labels, point_labels)
    println("*************Transition F1**************")
    println("BarT")
    println(bart_transition_recall, " ",bart_transition_precision, " ",bart_transition_F1)
    println("MF")
    println(mf_transition_recall," ", mf_transition_precision," ", mf_transition_F1)
    # cluster segments with DPC, return clustering and decision graph

    # generate figures
    fig = plot_trajectories_with_change_points(
        X,
        bart_cps, bart_point_labels, bart_positives, bart_negatives,
        mf_cps, mf_point_labels, mf_positives, mf_negatives,
        true_cps, point_labels
    )
    #save("toy-data-$(σ)-$(trans)-transition-ids.png", fig)
    println(size(mf_cps), size(bart_cps), size(true_cps))
    #println(abs.(mf_cps - true_cps))
    return fig
end



function write_distances(σ, trans)
    X = readdlm("../data/Toy_Model_Trajectories/sigma_$(σ)_transition_$(trans).txt")
    true_cps = convert(Array{Int32}, readdlm("../data/Toy_Model_Trajectories/sigma_$(σ)_transition_$(trans)_GroundTruth.txt"))
    bart_cps = convert(Array{Int32}, readdlm("../data/Toy_Model_Trajectories/sigma_$(σ)_transition_$(trans)_cps.txt"))
    true_segment_lenghts = [true_cps[i+1] - true_cps[i] + 1 for i=1:length(true_cps) - 1]
    windowsize = minimum(true_segment_lenghts[true_segment_lenghts .> maximum([trans 2])])
    quant = 0.5
    mf_deriv = compute_metric_wasserstein_derivative(X, windowsize, sqeuclidean)[:,1]
    mf_cps = compute_change_points(X, quant, windowsize)
    true_cps[1] = 1
    bart_cps[1] = 1

    mf_dists = pairwise_segment_distances_1d(X, mf_cps)
    bart_dists = pairwise_segment_distances_1d(X, bart_cps)
    true_dists = pairwise_segment_distances_1d(X, true_cps)
    open("/home/dcg/projects/DPA/toy_$(σ)_$(trans)_mf_dists.txt", "w") do io
        writedlm(io, mf_dists)
    end
    open("/home/dcg/projects/DPA/toy_$(σ)_$(trans)_true_dists.txt", "w") do io
        writedlm(io, true_dists)
    end
    open("/home/dcg/projects/DPA/toy_$(σ)_$(trans)_bart_dists.txt", "w") do io
        writedlm(io, bart_dists)
    end

    return (X, mf_cps, bart_cps, true_cps)
end

write_distances (generic function with 1 method)

In [6]:
σ = 5
trans = 50
tolerance = trans // 3
f = cp_pipeline(σ, trans; tolerance=tolerance)
X, mf_cps, bart_cps, true_cps = write_distances(σ, trans)
#save("toy-$(σ)-$(trans)-f1.png", f)

function draw_decision_graph(D; q=0.02, seg_lengths=nothing)
    N, _ = size(D)
    dc = quantile!([D[i,j] for i=1:N, j=1:N if i > j], q)
    if isnothing(seg_lengths)
        ρ = hcat(collect(1:N), [length(row[row .<= dc]) for row in eachrow(D)])
    else
        ρ = hcat(collect(1:N), [sum(seg_lengths .* exp.(-(D[i,:] / dc).^2)) for i in 1:N])
    end
    δ = zeros(N)
    nearest_denser_neighbor = zeros(N)
    for i=1:N
        indices = convert(Array{Int32},filter!(idx -> idx ∉ [i], ρ[ρ[:,2] .>= ρ[i, 2],:][:,1]))
        if length(indices) > 0
            δ[i] = minimum(D[indices, i])
            nearest_denser_neighbor[i] = argmin(D[indices, i])
        else
            δ[i] = maximum(D[:,i])
            nearest_denser_neighbor[i] = i
        end
    end
    γ = ρ[:,2] .* δ
    fig = Figure(size=(1000, 500))
    ax1 = Axis(fig[1,1], title="ρ-δ Plot", xlabel="ρ", ylabel="δ")
    ax2 = Axis(fig[1,2], title="γ Plot", xlabel="Rank", ylabel="γ")
    scatter!(ax1, ρ[:,2], δ)
    scatter!(ax2, 1:length(γ), sort!(γ))
    return (ax1, ax2, ax3)
end

function get_dp_stats(D; q=0.02, seg_lengths=nothing)
    N, _ = size(D)
    dc = quantile!([D[i,j] for i=1:N, j=1:N if i > j], q)
    if isnothing(seg_lengths)
        ρ = hcat(collect(1:N), [length(row[row .<= dc]) for row in eachrow(D)])
    else
        ρ = hcat(collect(1:N), [sum(seg_lengths .* exp.(-(D[i,:] / dc).^2)) for i in 1:N])
    end
    δ = zeros(N)
    nearest_denser_neighbor = zeros(N)
    for i=1:N
        indices = convert(Array{Int32},filter!(idx -> idx ∉ [i], ρ[ρ[:,2] .>= ρ[i, 2],:][:,1]))
        if length(indices) > 0
            δ[i] = minimum(D[indices, i])
            nearest_denser_neighbor[i] = argmin(D[indices, i])
        else
            δ[i] = maximum(D[:,i])
            nearest_denser_neighbor[i] = i
        end
    end
    γ = ρ[:,2] .* δ
    return (ρ, δ, γ)
end

bart_segment_lengths = [bart_cps[T+1] - bart_cps[T] for T in 1:length(bart_cps) - 1]
mf_segment_lengths = [mf_cps[T+1] - mf_cps[T] for T in 1:length(mf_cps) - 1]
true_segment_lengths = [true_cps[T+1] - true_cps[T] for T in 1:length(true_cps) - 1]

lengths = [true_segment_lengths, bart_segment_lengths, mf_segment_lengths]

fig = Figure(size=(900, 900))

title = ["Ground Truth", "BarT", "MF"]
dist_mat = [
    pairwise_segment_distances_1d(X, true_cps),
    pairwise_segment_distances_1d(X, bart_cps),
    pairwise_segment_distances_1d(X, mf_cps)
]

for i=1:3
    ax1 = Axis(fig[i, 1])
    ax2 = Axis(fig[i, 2])
    ax3 = Axis(fig[i, 3])

    ax1.title = "$(title[i]) ρ,δ"
    ax2.title = "$(title[i]) γ"
    ax3.title = "$(title[i]), logγ"
    ρ, δ, γ = get_dp_stats(dist_mat[i], seg_lengths=lengths[i])
    scatter!(ax1, ρ[:,2], δ)
    ax1.xlabel = "ρ"
    ax1.ylabel = "δ" 
    ax2.xlabel = "Rank"
    ax2.ylabel = "γ" 
    ax3.xlabel = "Rank"
    ax3.ylabel = "log γ"
    scatter!(ax2, 1:length(γ), sort!(γ))
    scatter!(ax3, 1:length(γ), log.(γ))
end

fig
save("toy-$(σ)-$(trans)-dc.png", fig)

103
0.5
enumerating change points
number of dimensions: 1
729.2263023232225
26.229754097208
30//
50//3
*************Change Point F1**************
BarT
1.0 0.7213114754098361 0.8380952380952381
MF
1.0 1.0 1.0
*************Transition F1**************
BarT
0.9336842105263158 0.8861138861138861 0.9092772936955408
MF
0.9147368421052632 0.9425162689804772 0.9284188034188035
(40, 1)
(40,)(61, 1)(40, 1)
enumerating change points
number of dimensions: 1
Number of segments = 91
Computing 4095 segment distances


Progress: 100%|█████████████████████████████████████████| Time: 0:00:00


finished computing distances
Number of segments = 60
Computing 1770 segment distances
finished computing distances
Number of segments = 39
Computing 741 segment distances
finished computing distances
Number of segments = 39
Computing 741 segment distances
finished computing distances
Number of segments = 60
Computing 1770 segment distances
finished computing distances
Number of segments = 91
Computing 4095 segment distances


Progress: 100%|█████████████████████████████████████████| Time: 0:00:00


finished computing distances


CairoMakie.Screen{IMAGE}


In [74]:
mf_seg_labels = convert(Array{Int32}, readdlm("/home/dcg/projects/DPA/toy_$(σ)_$(trans)_mf_dists_dpa_labels.txt"))[:,1]
bart_seg_labels = convert(Array{Int32}, readdlm("/home/dcg/projects/DPA/toy_$(σ)_$(trans)_bart_dists_dpa_labels.txt"))[:,1]
true_seg_labels = convert(Array{Int32}, readdlm("/home/dcg/projects/DPA/toy_$(σ)_$(trans)_true_dists_dpa_labels.txt"))[:,1]
mf_pt_labels = label_series(X, mf_cps, mf_seg_labels)
bart_pt_labels = label_series(X, bart_cps, bart_seg_labels)
true_pt_labels = label_series(X, true_cps, true_seg_labels)

f = plot_clustered_trajectories(X,
        bart_cps, bart_pt_labels, 
        mf_cps, mf_pt_labels,
        true_cps, true_pt_labels
    )
save("toy-$(σ)-$(trans)-dpa.png", f)

Dict(0.0 => 2521, 2.0 => 912, 1.0 => 2517)
Dict(0.0 => 2592, 2.0 => 763, 1.0 => 2595)
Dict(0.0 => 2499, 2.0 => 950, 1.0 => 2501)


CairoMakie.Screen{IMAGE}


Conclusion: in the toy trajectory model, with largest variance, and ground truth labeling, all of the metastable states retain the hypothesis that the samples were drawn from the MLE distribution.

In [62]:
segment_lengths = [bart_cps[T+1] - bart_cps[T] for T in 1:length(bart_cps) - 1]
idx = sortperm(segment_lengths, rev=true);
fig = Figure(size=(1500, 1500))
for i = 1:3
T = idx[i]
segment = X[bart_cps[T]:bart_cps[T+1]]
U = fit_mle(Normal, segment)
V = fit_mle(Laplace, segment)
W = fit_mle(Uniform, segment)
Uks = ExactOneSampleKSTest(segment, U)
Vks = ExactOneSampleKSTest(segment, V)
Wks = ExactOneSampleKSTest(segment, W)
t = collect(range(minimum(segment), maximum(segment), length=1000))
ax = Axis(fig[i,1], title="Kolmogorov-Smirnov Test for segment #$(T)\n# Samples: $(length(segment))\nHypothesis: Laplace Distribution\np-value: $(round(pvalue(Vks),digits=3))", ylabel="CDF")
ax2 = Axis(fig[i,2], title="Kolmogorov-Smirnov Test for segment #$(T)\n# Samples: $(length(segment))\nHypothesis: Normal Distribution\np-value: $(round(pvalue(Uks),digits=3))", ylabel="CDF")
ax3 = Axis(fig[i,3], title="Kolmogorov-Smirnov Test for segment #$(T)\n# Samples: $(length(segment))\nHypothesis: Uniform Distribution\np-value: $(round(pvalue(Wks),digits=3))", ylabel="CDF")
lines!(ax, t, cdf.(V,t), color="blue")
lines!(ax, sort(segment), (1:length(segment))./length(segment),color="red")
lines!(ax2, t, cdf.(U,t), color="blue")
lines!(ax2, sort(segment), (1:length(segment))./length(segment),color="red")
lines!(ax3, t, cdf.(W,t), color="blue")
lines!(ax3, sort(segment), (1:length(segment))./length(segment),color="red")
end
current_figure()
save("toy-ks-test-sigma$(σ)-tr$tr.png", fig)
#save("langevin-ks-test.pdf", fig)


CairoMakie.Screen{IMAGE}
